# best_three_players_algorithm

In [337]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.utils import check_array
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import warnings
import xgboost as xgb
import seaborn as sns
from sklearn import metrics
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [338]:
dataset = pd.read_csv('DBClassifiedFinal.csv')
dataset = dataset.drop(columns=['Unnamed: 0'])
invalid_idx = []
for ii in range(len(dataset)):
    x = dataset.iloc[ii].cast
    if (len(x) > 100) or x == '[]':
        invalid_idx.append(ii)
        
print("Drop all invalid rows:", len(invalid_idx))

dataset = dataset.drop(invalid_idx)
dataset = dataset.drop_duplicates()

dataset.to_csv('DBClassifiedFinal_Final_After_Cleaning.csv')
dataset

Drop all invalid rows: 135


,title,movie_id,cast,budget,revenue,producers,writers,directors
0,Pirates of the Caribbean: At World's End,285,17.32105912,3.000000,9.610000,10.237959,10.656598,10.656598
1,Spectre,206647,9.566367436666667,2.450000,8.806746,11.085610,10.760058,11.085610
2,The Dark Knight Rises,49026,10.870944316666666,2.500000,10.849391,10.849391,10.849391,10.849391
3,John Carter,49529,3.8206283033333333,2.600000,2.841391,5.447877,7.965731,9.403355
4,Spider-Man 3,559,12.089258133333333,2.580000,8.908716,8.908716,8.908716,8.908716
5,Tangled,38757,6.733775596666667,2.600000,5.917949,6.521054,5.917949,5.917949
6,Avengers: Age of Ultron,99861,15.195579099999998,2.800000,14.054037,15.195579,15.195579,15.195579
7,Harry Potter and the Half-Blood Prince,767,10.126300663333334,2.500000,9.339592,9.481313,9.764755,9.382127
8,Batman v Superman: Dawn of Justice,209112,10.842565826666666,2.500000,8.732602,9.790996,9.790996,8.732602
9,Superman Returns,1452,5.705783973333333,2.700000,3.910812,5.100084,5.439348,7.478628


In [339]:
dataset = pd.read_csv('DBClassifiedFinal_Final_After_Cleaning.csv')

x = dataset[['cast', 'budget', 'producers', 'writers', 'directors']].values
y = dataset['revenue'].values 
number_of_rows = len(dataset)
rand_gen = np.random.RandomState(0)
shuffled_indices = rand_gen.permutation(np.arange(len(x)))
print(shuffled_indices)
num_train = int(0.80 * number_of_rows)

[ 383 1448 1271 ... 1653 2607 2732]


In [340]:
# x = dataset[['cast', 'budget', 'writers', 'directors', 'producers']].values
# y = dataset['revenue'].values 

# external functions

In [341]:
# find all the triples with the given sum
def find_triplets(the_sum):
    arr = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10]
    n = len(arr)
    numbers = list()
    for i in range(0, n - 2):
        for j in range(i + 1, n - 1):
            for k in range(j + 1, n):
                if arr[i] + arr[j] + arr[k] == the_sum:
                    triple = [arr[i], arr[j], arr[k]]
                    triple.sort()
                    numbers.append(triple)
    ans = set(tuple(i) for i in numbers)
    return ans

In [342]:
# should be normalize for all
def normalize_budget(df_train, budget):
    budget = budget/100000000
    mean_f = float(df_train['budget'].mean())
    std_f = float(df_train['budget'].std())
    normalized_value = (budget - mean_f) / std_f
    print("normalized is:", normalized_value)
    print("mean and std are:", mean_f, std_f)
    difference = 0.135
    cur_range_min = 0.00001
    final_budget_value = 0
    for i in range(3, 31):
        if cur_range_min <= normalized_value < (cur_range_min + difference):
            final_budget_value = i
            break
        else:
            cur_range_min += difference
        if normalized_value > 3.8:  # kitzon
            final_budget_value = 30
            break
    print("final_budget_value is:",final_budget_value)
    return final_budget_value

In [343]:
def normalize_mean_std(df, df_train, feature_array):
    for feature_name in feature_array:
        mean_f = float(df_train[feature_name].mean())
        std_f = float(df_train[feature_name].std())
        df.loc[df.index, feature_name] = (df[feature_name] - mean_f) / std_f
    return df  

def normalize_mean_std_one_elememt(df_train, feature_name, feature_value):
    mean_f = float(df_train[feature_name].mean())
    std_f = float(df_train[feature_name].std())
    return (feature_value - mean_f) / std_f

df = normalize_mean_std(dataset, dataset.iloc[shuffled_indices[:num_train]], ['cast', 'budget','revenue', 'producers', 'writers', 'directors'])
df = df.drop(columns=['Unnamed: 0'])
df

,title,movie_id,cast,budget,revenue,producers,writers,directors
0,Pirates of the Caribbean: At World's End,285,3.429651,6.661728,5.597967,3.327030,3.731070,3.418613
1,Spectre,206647,1.150199,5.292420,5.082990,3.664657,3.774426,3.585371
2,The Dark Knight Rises,49026,1.533673,5.416903,6.392558,3.570569,3.811862,3.493552
3,John Carter,49529,-0.538732,5.665868,1.258517,1.419093,2.603422,2.931475
4,Spider-Man 3,559,1.891790,5.616075,5.148364,2.797579,2.998594,2.739207
5,Tangled,38757,0.317573,5.665868,3.230942,1.846550,1.745269,1.576689
6,Avengers: Age of Ultron,99861,2.804877,6.163798,8.447102,5.301698,5.633197,5.182926
7,Harry Potter and the Half-Blood Prince,767,1.314789,5.416903,5.424605,3.025650,3.357330,2.923223
8,Batman v Superman: Dawn of Justice,209112,1.525331,5.416903,5.035455,3.149000,3.368326,2.670751
9,Superman Returns,1452,0.015400,5.914833,1.944138,1.280564,1.544704,2.183328


# The algorithm

In [344]:
# The algorithm:
# Gets a budget, producer, director, writer and returns a tuple of the 3 major players that will give the best revenue
# for the movie in the budget restrictions
# the input of the producer, director, writer are strings with their names ?????????

In [345]:
def best_three_players_algorithm(rf_model, df_train, budget, producer, director, writer):
    # Vars:
    cur_revenue = 0
    max_revenue = 0
    max_player_1_name = ""
    max_player_2_name = ""
    max_player_3_name = ""

    # Normalize budget
    budget_value = normalize_budget(df_train, budget)

    # Get all the triples in the budget restrictions
    tuples_set = find_triplets(budget_value)
    print(tuples_set)

    # the files we need to work with:
    database = pd.read_csv('DBclassified.csv', low_memory=False)  # the file with the whole data
    # database_clean = pd.read_csv('DBClassifiedFinal_Final_After_Cleaning.csv', low_memory=False)
    actors_vals = pd.read_csv('ActorsGradesMax.csv', low_memory=False)
    actors_1 = pd.read_csv('Actors_1.csv', low_memory=False)
    actors_2 = pd.read_csv('Actors_2.csv', low_memory=False)
    actors_3 = pd.read_csv('Actors_3.csv', low_memory=False)
    actors_4 = pd.read_csv('Actors_4.csv', low_memory=False)
    actors_5 = pd.read_csv('Actors_5.csv', low_memory=False)
    actors_6 = pd.read_csv('Actors_6.csv', low_memory=False)
    actors_7 = pd.read_csv('Actors_7.csv', low_memory=False)
    actors_8 = pd.read_csv('Actors_8.csv', low_memory=False)
    actors_9 = pd.read_csv('Actors_9.csv', low_memory=False)
    actors_10 = pd.read_csv('Actors_10.csv', low_memory=False)

    # For each triple check:
    # Check how many times to get random values from each triple
    # Run for x times the random selection:
    # 1. Check if there are two players who participated in the same movie & the movie value is grater or equal to one
    #    (parameter to a success movie). If it is - add a bonus to this team of players.
    # calculate the average with or without the bonus.
    # 2. Send the cast value with the others to the regressor.
    # 3. keep the triple who has the best revenue value.
    max_triple = ()
    random_checks = 20
    for triple in tuples_set:
        for i in range(random_checks+1):
            actors_1th = vars()["actors_" + str(triple[0])]
            actors_2nd = vars()["actors_" + str(triple[1])]
            actors_3rd = vars()["actors_" + str(triple[2])]

            # get the rows of the actors
            player_1 = actors_1th.sample()
            player_2 = actors_2nd.sample()
            player_3 = actors_3rd.sample()

            print(player_1)
            print(player_2)
            print(player_3)

            # make sure there are not duplicate actors
            while player_1.equals(player_2):
                print("equal1")
                player_2 = actors_2nd.sample()
            while player_1.equals(player_3):
                print("equal2")
                player_3 = actors_3rd.sample()
            while player_2.equals(player_3):
                print("equal3")
                player_3 = actors_3rd.sample()

            # players info:
            player_1_id = player_1['ID'].iloc[0]
            player_1_name = player_1['Name'].iloc[0]
            player_2_id = player_2['ID'].iloc[0]
            player_2_name = player_2['Name'].iloc[0]
            player_3_id = player_3['ID'].iloc[0]
            player_3_name = player_3['Name'].iloc[0]

            # check for bonus
            bonus = 0
            counter = 1
            for movie in database.iterrows():
                counter += 1
                if counter == 61 or counter == 1737 or counter == 2078 or counter == 2348 or counter == 2608 or counter == 2779 or counter == 2791:
                    continue
                if counter == 3142 or counter == 3230 or counter == 3292 or counter == 3398 or counter == 3477 or counter == 3679 or counter == 3776:
                    continue
                if counter == 3905 or counter == 4000 or counter == 4017 or counter == 4076 or counter == 4113 or counter == 4126 or counter == 4148:
                    continue
                if counter == 4185 or counter == 4255 or counter == 4312 or counter == 4321 or counter == 4329 or counter == 4335 or counter == 4394:
                    continue
                if counter == 4408 or counter == 4438 or counter == 4465 or counter == 4475 or counter == 4478 or counter == 4498 or counter == 4510:
                    continue
                if counter == 4511 or counter == 4515 or counter == 4524 or counter == 4533 or counter == 4547 or counter == 4557 or counter == 4560:
                    continue
                if counter == 4569 or counter == 4571 or counter == 4573 or counter == 4578 or counter == 4581 or counter == 4588 or counter == 4590:
                    continue
                if counter == 4596 or counter == 4604 or counter == 4613 or counter == 4623 or counter == 4624 or counter == 4640 or counter == 4645:
                    continue
                if counter == 4664 or counter == 4665 or counter == 4681 or counter == 4686 or counter == 4692 or counter == 4696 or counter == 4705:
                    continue
                if counter == 4717 or counter == 4719 or counter == 4723 or counter == 4726 or counter == 4744 or counter == 4762 or counter == 4764:
                    continue
                if counter == 4786 or counter == 4789 or counter == 4804:
                    continue
                curr_movie_players = []
                curr_movie_id = movie[1][2]
                #print("curr movie id is:", curr_movie_id)
                
                # df is the normalized file 
                movie_value = 0
                if curr_movie_id in df.movie_id.values:
                    movie_value = (df.loc[df.movie_id == curr_movie_id, 'revenue']).values[0]
                #print("movie value is:" ,movie_value)
                
                if movie_value > 1: # a parameter to a success movie (when the revenue is grater then 1 after normalization)
                    # print("checking for 2 who participated in same movie")
                    cast = movie[1][3]
                    array = cast.split(',')
                    curr_movie_players.append(array[4].split(': ')[1])
                    curr_movie_players.append(array[4+7].split(': ')[1])
                    curr_movie_players.append(array[4+14].split(': ')[1])
                    same_movie = 0
                    if player_1_id in curr_movie_players:
                        same_movie += 1
                    if player_2_id in curr_movie_players:
                        same_movie += 1
                    if player_3_id in curr_movie_players:
                        same_movie += 1
                    if same_movie >= 2:
                        print("same movie there is bonus!!!!!!!!!!")
                        bonus = 3
                        break

            # calculate actors average
            val1 = (actors_vals.loc[actors_vals.ID == player_1_id, 'Grade']).values[0]
            val2 = actors_vals.loc[actors_vals.ID == player_2_id, 'Grade'].values[0]
            val3 = actors_vals.loc[actors_vals.ID == player_3_id, 'Grade'].values[0]
            actors_avg = (val1 + val2 + val3 + bonus)/3
            # initial bonus
            bonus = 0
            
            # make the test example to the regressor
            # the order: cast, budget, producer, director, writer
            actors_value = normalize_mean_std_one_elememt(df_train, 'cast', actors_avg)
            producer_value = normalize_mean_std_one_elememt(df_train, 'producers', producer)
            director_value = normalize_mean_std_one_elememt(df_train, 'directors', director)
            writer_value = normalize_mean_std_one_elememt(df_train, 'writers', writer)
            test_to_check = np.array([actors_value, budget_value, producer_value, director_value, writer_value])

            # check on the regressor
            cur_revenue = rf_model.predict(test_to_check.reshape(1, -1))
            print("revenue is:", cur_revenue)
            print("-----------------------------------------------------------------------------")
            print("-----------------------------------------------------------------------------")

            # update max triple:
            if cur_revenue > max_revenue:
                max_revenue = cur_revenue
                max_player_1_id = player_1_id
                max_player_1_name = player_1_name
                max_player_2_id = player_2_id
                max_player_2_name = player_2_name
                max_player_3_id = player_3_id
                max_player_3_name = player_3_name

        # initial random checks
        random_checks = 20
        # move to next triple

    # return the best triple
    return max_player_1_name, max_player_2_name, max_player_3_name

# Run the algorithm

In [346]:
x = df[['cast', 'budget', 'producers', 'writers', 'directors']].values
y = df['revenue'].values 

rand_gen = np.random.RandomState(0)
shuffled_indices = rand_gen.permutation(np.arange(len(x)))
print(shuffled_indices)
num_train = int(0.8 * number_of_rows)

x_train = x[shuffled_indices[:num_train]]
y_train = y[shuffled_indices[:num_train]]
x_test = x[shuffled_indices[num_train:]]
y_test = y[shuffled_indices[num_train:]]

# train the model:
rf_model = RandomForestRegressor(n_estimators=500, random_state=42)
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_test)

[ 383 1448 1271 ... 1653 2607 2732]


In [347]:
# external function to the Algorithm:
# gets the producers, directors and writers by names in a list

    
def calculate_avrg_value_from_list(job_list, dataframe_grades):
    value = 0
    for person in job_list:
        print(person)
        cur_person = " \"" + person + "\""   
        print(cur_person)
        print((dataframe_grades.loc[dataframe_grades.ID == 2710, 'Name']).values[0])
        
        person_val = (dataframe_grades.loc[dataframe_grades.Name == cur_person, 'Grade']).values[0]
        value += person_val
        
    return value/len(job_list)
        

def external_best_three_players_algorithm(rf_model, df_train, budget, producers_list, directors_list, writers_list):
    producers_grades = pd.read_csv('ProducersGrades.csv', low_memory=False)
    directors_grades = pd.read_csv('DirectorsGrades.csv', low_memory=False)
    writers_grades = pd.read_csv('WritersGrades.csv', low_memory=False)
    
    producers_val = calculate_avrg_value_from_list(producers_list, producers_grades)
    directors_val = calculate_avrg_value_from_list(directors_list, directors_grades)
    writers_val = calculate_avrg_value_from_list(writers_list, writers_grades)
    
    print(producers_val,directors_val,writers_val)
    return best_three_players_algorithm(rf_model, dataset.iloc[shuffled_indices[:num_train]], budget, producers_val, directors_val, writers_val)
     
    

In [348]:
# def best_three_players_algorithm(rf_model, df_train, budget, producer, director, writer)

# x = best_three_players_algorithm(rf_model, dataset.iloc[shuffled_indices[:num_train]], 10000000, 10, 6.5, 5.5)
# print(x)

producers_list = ["James Cameron","Jon Landau"]
directors_list = ["James Cameron"]
writers_list = ["Ted Elliott", "John Logan"]

x = external_best_three_players_algorithm(rf_model, dataset.iloc[shuffled_indices[:num_train]], 10000000, producers_list, directors_list, writers_list)

James Cameron
 "James Cameron"
 "James Cameron"
Jon Landau
 "Jon Landau"
 "James Cameron"
James Cameron
 "James Cameron"
 "James Cameron"
Ted Elliott
 "Ted Elliott"
 "James Cameron"
John Logan
 "John Logan"
 "James Cameron"
18.45034188 18.45034188 10.871104124999999
normalized is: 0.09999999999999751
mean and std are: 2.0780491710852045e-15 1.0000000000000042
final_budget_value is: 3
{(1, 1, 1)}
     OriginalOrder      ID              Name     Grade  Value
802           1608  137920  "Hanaa Bouchaib"  0.251478      1
     OriginalOrder     ID              Name     Grade  Value
778           1583  20402  "Julian McMahon"  0.841468      1
     OriginalOrder     ID                Name     Grade  Value
823           1636  56890  "Malcolm McDowell"  0.222114      1
revenue is: [6.87466839]
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
      OriginalOrder     ID           Name  Grade

revenue is: [6.83225327]
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
      OriginalOrder       ID            Name  Grade  Value
4155           5139  1428379  "Shea Stewart"   0.01      1
      OriginalOrder     ID            Name     Grade  Value
2753           3700  74940  "Lumi Cavazos"  0.005601      1
      OriginalOrder     ID                 Name     Grade  Value
1125           1989  52869  "Madison Davenport"  0.854461      1
revenue is: [6.8633931]
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
      OriginalOrder     ID           Name  Grade  Value
1821           2739  21485  "Blake Clark"  0.167      1
      OriginalOrder    ID            Name    Grade  Value
3345           4307  8841  "Donald Crisp"  0.09132      1
      OriginalOrder     ID           Name  